# Módulo 1: Análisis de datos en el ecosistema Python

### Sesión (23)

## Aplicar un caso de uso (*Clustering*)

### Datset de Bike-sharing

Uno de los conjuntos de datos reales e interesantes para ejercicios de _regresión_, _clasificación_ y también ___clustering___ es el **Conjunto de datos para compartir bicicletas** o el __[Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset)__ de la _Universidad de Oporto_. Este dataset contiene datos de uso de bicicletas de alquiler de un programa de bicicletas compartidas en _Washington D.C._ durante un período de dos años (_2011_ y _2012_).

Uno de los retos que se plantea para este conjunto de datos es **agrupar los datos de _bike-sharing_** en función de las diferentes **condiciones climáticas** y otros factores, como **días festivos y laborales**, para **`identificar patrones en el uso de bicicletas de alquiler`**. Esto puede ayudar a las empresas de _bike-sharing_ a **comprender mejor el comportamiento de los clientes** y el tipo de viajes realizados con el fin de **optimizar** sus servicios. Además, se puede analizar los **patrones de movilidad en la ciudad** de cara a la **gestión eficiente** de los servicios municipales.

Este dataset incluye 17 variables para los datos horarios y ***16 variables*** para el conjunto de datos agregados a nivel ***diario***:

- **instant**:  Índice de registro  
- **dteday**:  Fecha y hora de la observación
- **season**:  Estación del año (1:invierno, 2:primavera, 3:verano, 4:otoño)
- **yr**:  Año (0:2011, 1:2012)
- **mnth**:  Mes (1 a 12)
- **holiday**:  Si el día es festivo o no
- **weekday**:  Día de la semana (0 a 6)
- **workingday**:  Si el día es laborable o no (1: sí, 0: no)
- **weathersit**:  Situación meteorológica (1: despejado, 2: nublado, 3: lluvia ligera/nieve, 4: lluvia intensa/nieve)
- **temp**:  Temperatura (normalizada en Celsius)
- **atemp**:  Temperatura marcada por la sensación térmica (normalizada en Celsius)
- **hum**:  Humedad normalizada
- **windspeed**:  Velocidad del viento normalizada
- **casual**:  Número de usuarios casuales (no registrados)
- **registrado**:  Número de usuarios registrados
- **cnt**:  Recuento total de bicicletas alquiladas (suma de usuarios casuales y registrados).



In [ ]:
# importamos las librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# Modificamos los parámetros de los gráficos en matplotlib
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 18, 8 # el primer dígito es el ancho y el segundo el alto
rcParams["font.weight"] = "bold"
rcParams["font.size"] = 10
rcParams["axes.labelweight"] = "bold"

Leemos los datos mediante el archivo adjunto.

In [ ]:
# Lectura de datos
df_bike = pd.read_csv('bike-sharing-daily.csv')
df_bike

Vamos a quitar los datos que no son de intrés para este estudio, como el número de registros o la fecha que en sí no nos dice nada. Nos quedamos con el número total de bicicletas sin desglosarlo.

In [ ]:
col_drop = ['instant', 'dteday', 'yr', 'casual', 'registered']
df_bike2 = df_bike.drop(columns=col_drop)
df_bike2

In [ ]:
# La información útil sobre los datos guardados en formato DataFrame
df_bike2.info()

Podemos ver que **no hay valores nulos** y toda la información es **numérica**.

In [ ]:
# Echamos un vistazo a las características de cada variable
df_bike2.describe()

Es evidente que la escala de los datos es muy variada y por ello más adelante procederemos a estandarizar los datos.  

#### Ánalisis exploratorio

Vamos a realizar una serie de gráficas con el objetivo de **conocer mejor los datos y la información** que contienen estas muestras.

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15, 6))

# Count plot (primera gráfica)
sns.countplot(data=df_bike2, x='workingday', label=df_bike2['workingday'].value_counts().sort_values().index, palette=['#5cde59',"#de5458"], ax=axes[0])
axes[0].set_title("Count plot")
axes[0].set_ylabel("Count")

# pie chart (segunda gráfica)
plt.pie(df_bike2['workingday'].value_counts().sort_values(), autopct='%.1f%%', labels=['No Laboral', 'Laboral'], colors=['#5bde54',"#de5454"])
axes[1].set_title("Pie chart")

plt.show()

La variable más determinante en este análisis sería el **número de los usuarios/viajes** que se han registrado cada día. Realizamos un análisis estadístico para ver la **distribución de volumetría de bicicletas alquiladas** en función de otras variables del dataset.   

In [ ]:
# Análisis de volumetría en función de días laborables
sns.boxplot(data=df_bike2, x='workingday', y='cnt')
plt.show()

In [ ]:
# Análisis de volumetría en función de la festividad
sns.boxplot(data=df_bike2, x='holiday', y='cnt')
plt.show()

In [ ]:
# Análisis de volumetría en función de días la semana
sns.boxplot(data=df_bike2, x='weekday', y='cnt')
plt.show()

In [ ]:
# Análisis de volumetría por mes
sns.boxplot(data=df_bike2, x='mnth', y='cnt')
plt.show()

Este gráfico confirma que nuestro dataset tiene una **estacionalidad considerable**. Este hecho seguramente influiría en la extracción de grupos de viajes y analizar los **patrones de movilidad** correspondiente al servicio de _bike-sharing_.

In [ ]:
# Análisis de volumetría en función de la estación del año
sns.boxplot(data=df_bike2, x='season', y='cnt')
plt.show()

Es de esperar que la estacionalidad mensual afecte en su forma agregada a la **evolución trimestral** de volumetría de _bike-sharing_ según la **estación del año**.

In [ ]:
# Análisis de volumetría en función de la situación climática
sns.boxplot(data=df_bike2, x='weathersit', y='cnt')
plt.show()

Se observa una **distinción lógica** entre las volumetrías registradas para cada situación climática, indicando la importancia de las **condiciones meteorológicas** sobre el uso de servicio de _bike-sharing_.

Examinamos las realciones que hay entre las variables de tiempo para analizar mejor esta correlación.

In [ ]:
# Dibujar la relación entre la temperatura (la sensación térmica) y la volumetría
sns.scatterplot(data=df_bike2, x='atemp', y='cnt')
plt.show()

Se puede observar que efectivamente los días con **más bicicletas alquiladas** corresponden a los días con **las temperaturas moderadas**.

In [ ]:
sns.scatterplot(data=df_bike2, x='atemp', y='cnt', hue='workingday')
plt.show()

Si pintamos según los días laborables cada punto en la gráfica antreior entre temperatura y los volúmenes de viajes o bicicletas alquiladas, se ve que **no hay una realción** que distinga la evolución de los datos en función de esta variable.

In [ ]:
sns.scatterplot(data=df_bike2, x='atemp', y='cnt', hue='weathersit')
plt.show()

Pintando los datos de acuerdo con la calidad de tiempo que hace en la ciudad, muestra que los **días con menos usos** corresponden a **situaciones meteorológicas adversas**. Podemos observar este mismo comporertamiento si visualizamos las **humedades** registradas.

In [ ]:
sns.scatterplot(data=df_bike2, x='hum', y='cnt', hue='weathersit')
plt.show()

Viendo las gráficas anteriores que visualizan las diferentes variables meteorológicas presentes en este conjunto de datos, podemos quedarnos con la variable categórica de `weathersit` (_Weather situation_) que al parecer resume y representa la noción de cuando hace **"buen tiempo"** o no.

Revisamos otra vez la ditribución de volumetría de bicicletas alquiladas en función del día de la semana (_weekday_), **por si vemos que no aporta una información discriminatoria** a la hora de agrupar los patrones de uso de _bike-sharing_.

In [ ]:
sns.violinplot(data=df_bike2, x='weekday', y='cnt')
plt.show()

Al parecer podemos **prescindir de la variable `weekday`**, porque la **forma y las características** que tiene la distribución de valores para cada día son bastante **similares**.

Por otra parte, pasa algo parecido con la variable `workingday` que no tiene una distribución diferente en sus dos categorías y podemos **ver reflejado el efecto de los días laborales** quizá mejor en la variable **`holiday`**.

In [ ]:
fig , axes = plt.subplots(2, 1, figsize=(14,10))

sns.boxplot(data=df_bike2, x='workingday', y='cnt', ax=axes[0])
sns.boxplot(data=df_bike2, x='holiday', y='cnt', ax=axes[1])
plt.show()

Ahora hacemos otro filtrado para **reducir la dimensionalidad** del conjunto de datos y quedarnos con la parte **representativa** e interesante de los datos.

In [ ]:
df_bike3 = df_bike2.drop(columns=['weekday', 'workingday', 'temp', 'atemp', 'hum', 'windspeed'])
df_bike3

Analizamos en su conjunto las **relaciones exsitentes** entre las variables del dataset filtrado, considerando la importancia de las **condiciones meteorológicas**.

In [ ]:
sns.pairplot(df_bike3, hue='weathersit', height=3)
plt.show()

Esta vista sobre el conjunto de las variables **confirma el efecto discriminador de la variable `weathersit`**, ya que extrae **distribuciones desemejantes** en función de sus valores para cada variable. Hacemos zoom por ejemplo para visualizar mejor este efecto en la relación entre la volumetría de las bicicletas alquiladas diariamente y los meses del año considerando la situación del tiempo.

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=df_bike3, x='cnt', y='mnth', hue='weathersit')
plt.show()

---

### **`Ejercicio 23.1`**

Para poder realizar un ejercicio de **aprendizaje no-supervisado** tipo **clustering**, necesitamos hacernos primero una idea sobre el **número de los clusters o grupos** que están presentes en los datos.

**`23.1.1`** En primer lugar **estandariza** los datos del dataset filtrado (_df_bike3_) llevando los valores de cada variable a una escala **entre 0 y 1**.

**`23.1.2`** Utiliza las distintas técnicas de análisis de calidad de clustering y **estima según cada métrica** el número de los grupos de datos presentes en este data set:

 - WSS (Elbow method)
 - Silhouette score
 - Davies-Bouldin index
 - Calinski and Harabasz score
 - BIC score (implementación adhoc de la sesión 22)
 - BIC score of GMM
 - `random_state=100`

### **`Ejercicio 23.2`**

**`23.2.1`** Aplica un análisis de componentes principales usando el método **`PCA`** de la librería _sklearn_ para llevar los datos a un **espacio bidimensional**. Calcula **el porcentaje total de varianza** que incluyen los dos componentes principales juntos, después:

- Visualiza en una gráfica los datos del vector con los valores proyeccionados a los dos componentes principales.
- Visualiza la misma gráfica, ésta vez pintando los puntos de datos según la condición meteorológica del día (variable `weathersit`).

**`23.2.2`** Utiliza la visualización obtenida de los datos en el espacio bidimensional para hacer una **estimación del número óptimo de los clusters** o grupos presentes en el dataset. Y comprueba si coincide por ejemplo con el número estimado mediante la última técnica que aplicaste en el punto anterior (_23.1.2_)

**`23.2.3`** Vuelve a calcular las diferentes métricas de análisis de calidad de clustering, ésta vez usando el **vector con los datos proyeccionados** a los dos componentes principales, y compara si cada estimación coincide con el número óptimo de clusters inferido en el paso anterior:

 - WSS (Elbow method)
 - Silhouette score
 - Davies-Bouldin index
 - Calinski and Harabasz score
 - BIC score (implementación adhoc de la sesión 22)
 - BIC score of GMM
 - `random_state=100`

### **`Ejercicio 23.3`**

**`23.3.1`** Crea diferentes modelos de clustering **usando el vector con los datos proyeccionados a los dos componentes principales**, genera las gráficas de cada modelo dibujando los centroides si se corresponde y compara los resultados obtenidos según estas gráficas. Explica cuál de estos método logra **acercarse más a la agrupación sugerida** mediante la visualización de los datos transformados al espacio bidimensional (_23.2.2_) :  

 - `random_state=100`
 - K-Means
 - K-Medoids
 - GMM
 - DBSCAN (`eps=0.10`)
 - Agglomerative con dendograma (`method ='single', metric='euclidean'`)

**`23.3.2`** Añade los resultados del último modelo (_AgglomerativeClustering_) a los datos del DataFrame filtrado (_df_bike3_) para guardar en una nueva columna llamada `'labels'` las etiquetas generadas por el modelo de clustering jerárquico. Después, crea las siguientes gráficas para analizar estos resultados:

- _Count plot_ de los`'labels'` que muestra el tamaño de cada cluster.
- La distribución de la volumetría de viajes/bicicletas alquiladas al día (`'cnt'`) para cada cluster (por `'labels'` y usando _boxplot_).
- La distribución del mes de año (`'mnth'`) para cada cluster (por `'labels'` y usando _boxplot_).
- La distribución de la calidad del tiempo que hace en la ciudad (`'weathersit'`) para cada cluster (por `'labels'` y usando _boxplot_).
- La distribución de la estación del año (`'season'`) para cada cluster (por `'labels'` y usando _boxplot_).


### **`Ejercicio 23.4`**

**`23.4.1`** Compara las cracterísticas del **primer cluster** con el **décimo grupo** (_cluster_0_ y _cluster_9_) y explica los aspectos que revelan acada uno de estos grupos sobre los **distintos patrones de movilidad y el uso de _bike-sharing_** en la ciudad. Te puedes basar en los siguientes criterios para describir y comparar el perfil que representa cada cluster:

- El volúmen de los viajes diarios o el uso registrado de _bike-sharing_ por día.
- El mes de año.
- Las condiciones del tiempo.

Y al final sugiere algunos usos que se pueda sacar de estos resultados!!

---